In [1]:
import json
import cv2
from os.path import join as pjoin
from tqdm import tqdm

In [2]:
def draw_bounding_box(org, corners, color=(0, 255, 0), line=2, show=False):
    """
    Draw bounding box of components on the original image
    :param org: original image
    :param corners: [(top_left, bottom_right)]
                    -> top_left: (column_min, row_min)
                    -> bottom_right: (column_max, row_max)
    :param color: line color
    :param line: line thickness
    :param show: show or not
    :return: labeled image
    """
    board = org.copy()
    for i in range(len(corners)):
        board = cv2.rectangle(board, (corners[i][0], corners[i][1]), (corners[i][2], corners[i][3]), color, line)
    if show:
        cv2.imshow('a', cv2.resize(board, (300, 600)))
        key = cv2.waitKey(0)            
    return board, key


def load_ground_truth(annotation_file, data_file):
    def get_img_by_id(img_id):
        for image in images:
            if image['id'] == img_id:
                return image['file_name'].split('/')[-1][:-4], (image['height'], image['width'])

    def cvt_bbox(bbox):
        '''
        :param bbox: [x,y,width,height]
        :return: [col_min, row_min, col_max, row_max]
        '''
        bbox = [int(b) for b in bbox]
        return [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]

    data = json.load(open(data_file, 'r'))
    images = data['images']
    compos = {}
    annots = json.load(open(annotation_file, 'r'))
    print(len(annots))
    for annot in tqdm(annots):
        img_name, size = get_img_by_id(annot['image_id'])
        if img_name not in compos:
            compos[img_name] = {'bboxes': [cvt_bbox(annot['bbox'])], 'categories': [annot['category_id']], 'size':size}
        else:
            compos[img_name]['bboxes'].append(cvt_bbox(annot['bbox']))
            compos[img_name]['categories'].append(annot['category_id'])
    return compos


def draw_gt(gts, root):
    for img_id in gts:
        img_file = pjoin(root, str(img_id) + '.jpg')
        img = cv2.imread(img_file)
        _, key = draw_bounding_box(img, gts[img_id]['bboxes'], show=True)

In [3]:
ground_truth = load_ground_truth('E:\Temp\detections_val_results.json', 'E:\Mulong\Datasets\\rico/instances_val.json')

  0%|          | 548/503300 [00:00<01:32, 5440.40it/s]

503300


100%|██████████| 503300/503300 [01:22<00:00, 6133.80it/s]


In [4]:
draw_gt(ground_truth, 'E:\\Mulong\\Datasets\\rico\\combined')

KeyboardInterrupt: 